In [5]:
import rasterio
from rasterio.warp import reproject, Resampling
import numpy as np

# Define LULC columns and their remapped names
lulc_columns_remap = ['water', 'trees', 'flooded_veg', 'crop', 'built_area', 'bare_ground', 'range_land']

# Input file paths
lulc_path = "ESRI_LULC_2023_Rome.tif"  # LULC raster file path
lst_path = "../ecostress_LST/ECO2LSTE.001_SDS_LST_doy2024181064258_aid0001.tif"  # LST raster file path
output_path = "ESRI_LULC_2023_Rome_PROFILE_70m.tif"  # Output raster file path

# Read LULC raster and LST raster
with rasterio.open(lulc_path) as lulc_raster, rasterio.open(lst_path) as lst_raster:
    # Calculate the intersection of bounding boxes
    left = max(lulc_raster.bounds.left, lst_raster.bounds.left)
    bottom = max(lulc_raster.bounds.bottom, lst_raster.bounds.bottom)
    right = min(lulc_raster.bounds.right, lst_raster.bounds.right)
    top = min(lulc_raster.bounds.top, lst_raster.bounds.top)
    intersection = (left, bottom, right, top)

    # Crop to the overlapping area
    lulc_window = rasterio.windows.from_bounds(*intersection, transform=lulc_raster.transform)
    lst_window = rasterio.windows.from_bounds(*intersection, transform=lst_raster.transform)

    lulc_data = lulc_raster.read(1, window=lulc_window)  # Read single band
    lst_data = lst_raster.read(1, window=lst_window)

    # Update profiles for further processing
    lulc_profile = lulc_raster.profile
    lst_profile = lst_raster.profile

# Resample LULC to match LST resolution
resampled_lulc = np.empty_like(lst_data, dtype=np.int32)
reproject(
    lulc_data,
    resampled_lulc,
    src_transform=lulc_profile["transform"],
    dst_transform=lst_profile["transform"],
    src_crs=lulc_profile["crs"],
    dst_crs=lst_profile["crs"],
    resampling=Resampling.nearest  # Use nearest neighbor for categorical data
)

# Calculate proportions for each LULC class
unique_classes = np.unique(resampled_lulc)
proportions = {}


In [6]:

for class_value in unique_classes:
    # Create a binary mask for the current class
    mask = (resampled_lulc == class_value).astype(np.float32)
    proportions[class_value] = mask  # Store the binary mask as a proportion


In [7]:
# Remap class values to LULC column names
class_mapping = {i + 1: name for i, name in enumerate(lulc_columns_remap)}
remapped_proportions = {
    class_mapping[class_value]: proportions[class_value]
    for class_value in unique_classes if class_value in class_mapping
}

In [ ]:
remapped_proportions

In [11]:
# Update the profile with the correct number of bands
lst_profile.update(count=len(remapped_proportions))

# Write output raster
with rasterio.open(output_path, 'w', **lst_profile) as dst:
    for idx, (band_name, data) in enumerate(remapped_proportions.items(), start=1):
        dst.write_band(idx, data.astype(rasterio.float32))  # Ensure data type matches


In [16]:
import rasterio
from rasterio.warp import reproject, Resampling
import numpy as np

# Define LULC columns and their remapped names
lulc_columns_remap = ['water', 'trees', 'flooded_veg', 'crop', 'built_area', 'bare_ground', 'range_land']

# Input file paths
lulc_path = "ESRI_LULC_2023_Rome.tif"  # LULC raster file path
lst_path = "../ecostress_LST/ECO2LSTE.001_SDS_LST_doy2024181064258_aid0001.tif"  # LST raster file path
output_path = "ESRI_LULC_2023_Rome_PROFILE_70m_mb.tif"  # Output raster file path

# Read LULC raster and LST raster
with rasterio.open(lulc_path) as lulc_raster, rasterio.open(lst_path) as lst_raster:
    # Calculate the intersection of bounding boxes
    left = max(lulc_raster.bounds.left, lst_raster.bounds.left)
    bottom = max(lulc_raster.bounds.bottom, lst_raster.bounds.bottom)
    right = min(lulc_raster.bounds.right, lst_raster.bounds.right)
    top = min(lulc_raster.bounds.top, lst_raster.bounds.top)
    intersection = (left, bottom, right, top)

    # Crop to the overlapping area
    lulc_window = rasterio.windows.from_bounds(*intersection, transform=lulc_raster.transform)
    lst_window = rasterio.windows.from_bounds(*intersection, transform=lst_raster.transform)

    lulc_data = lulc_raster.read(1, window=lulc_window)  # Read single band
    lst_data = lst_raster.read(1, window=lst_window)

    # Update profiles for further processing
    lulc_profile = lulc_raster.profile
    lst_profile = lst_raster.profile

# Resample LULC to match LST resolution
resampled_lulc = np.empty_like(lst_data, dtype=np.int32)
reproject(
    lulc_data,
    resampled_lulc,
    src_transform=lulc_profile["transform"],
    dst_transform=lst_profile["transform"],
    src_crs=lulc_profile["crs"],
    dst_crs=lst_profile["crs"],
    resampling=Resampling.nearest  # Use nearest neighbor for categorical data
)

# Calculate proportions for each LULC class
unique_classes = np.unique(resampled_lulc)

print(unique_classes)


[ 0  1  2  5  7  8 11]


In [17]:

proportions = []

for class_value in unique_classes:
    # Create a binary mask for the current class
    mask = (resampled_lulc == class_value).astype(np.float32)
    proportions.append(mask)  # Store each class's mask as a separate band

# Stack proportions into a 3D array (bands, rows, cols)
proportions_stack = np.stack(proportions, axis=0)

# Update the profile for the output raster
lst_profile.update(count=proportions_stack.shape[0], dtype=rasterio.float32)


In [19]:
proportions_stack

array([[[1., 1., 1., ..., 1., 1., 1.],
        [1., 1., 1., ..., 1., 1., 1.],
        [1., 1., 1., ..., 1., 1., 1.],
        ...,
        [0., 0., 0., ..., 1., 1., 1.],
        [0., 0., 0., ..., 1., 1., 1.],
        [0., 0., 0., ..., 1., 1., 1.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [1., 1., 1., ..., 0., 0., 0.],
        [1., 1., 1., ..., 0., 0., 0.],
        [1., 1., 1., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       ...,

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0.

In [ ]:

# Write output multiband raster
with rasterio.open(output_path, 'w', **lst_profile) as dst:
    for idx in range(proportions_stack.shape[0]):
        dst.write(proportions_stack[idx, :, :], idx + 1)  # Write each band
